In [130]:
import pandas as pd
import boto3
import warnings
import os
from dotenv import load_dotenv
warnings.filterwarnings("ignore")
from io import StringIO

In [59]:
load_dotenv('env.s3creds')

True

In [60]:
s3 = boto3.resource(
	service_name='s3',
	aws_access_key_id=os.getenv('aws_access_key_id'),
    aws_secret_access_key=os.getenv('aws_secret_access_key'),
    region_name=os.getenv('region_name')
)

In [61]:
# List all buckets
for bucket in s3.buckets.all():
    print(bucket.name)

prj4


In [133]:
for obj in s3.Bucket('prj4').objects.all():
    print(obj)

s3.ObjectSummary(bucket_name='prj4', key='StormEvents_2014.csv')
s3.ObjectSummary(bucket_name='prj4', key='eaglei_outages_2014.csv')
s3.ObjectSummary(bucket_name='prj4', key='stormOutages_df.csv')


In [116]:
outages = s3.Bucket('prj4').Object('eaglei_outages_2014.csv').get()
outages_file = pd.read_csv(outages['Body'], index_col=0)

outages_df = pd.DataFrame(outages_file)

stormEvents = s3.Bucket('prj4').Object('StormEvents_2014.csv').get()
stormEvents_file = pd.read_csv(stormEvents['Body'], index_col=0)

stormEvents_df = pd.DataFrame(stormEvents_file)

In [117]:
outages_df['run_start_time'] = pd.to_datetime(outages_df['run_start_time'])
outages_df['yearMonth'] = outages_df['run_start_time'].dt.strftime("%Y%m")
outages_df = outages_df.reset_index(drop=False)
outages_df.head(5)

,fips_code,county,state,customers_out,run_start_time,yearMonth
0,1037,Coosa,Alabama,12,2014-11-01 04:00:00,201411
1,1051,Elmore,Alabama,7,2014-11-01 04:00:00,201411
2,1109,Pike,Alabama,1,2014-11-01 04:00:00,201411
3,1121,Talladega,Alabama,31,2014-11-01 04:00:00,201411
4,4017,Navajo,Arizona,1,2014-11-01 04:00:00,201411


In [118]:
outages_df = outages_df.astype({
    'fips_code' : str,
    'yearMonth' : str})

outages_df['FIPS'] = outages_df['fips_code'] + outages_df['yearMonth'] 
outages_df.head(5)



,fips_code,county,state,customers_out,run_start_time,yearMonth,FIPS
0,1037,Coosa,Alabama,12,2014-11-01 04:00:00,201411,1037201411
1,1051,Elmore,Alabama,7,2014-11-01 04:00:00,201411,1051201411
2,1109,Pike,Alabama,1,2014-11-01 04:00:00,201411,1109201411
3,1121,Talladega,Alabama,31,2014-11-01 04:00:00,201411,1121201411
4,4017,Navajo,Arizona,1,2014-11-01 04:00:00,201411,4017201411


In [119]:
stormEvents_df = stormEvents_df.reset_index(drop=False)
stormEvents_df.head(5)

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,201402,18,1000,201402,18,2000,83473,503953,NEW HAMPSHIRE,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Low pressure developing south of Long Island a...,Eight to twelve inches of snow fell across eas...,CSV
1,201402,5,300,201402,5,2300,83491,504065,NEW HAMPSHIRE,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Low pressure moving off the mid-Atlantic coast...,Six to twelve inches of snow fell across easte...,CSV
2,201401,18,1000,201401,19,700,82185,494521,NEW HAMPSHIRE,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Low pressure brought a brief period of heavy s...,Four to eight inches of snow fell across easte...,CSV
3,201411,26,1000,201411,27,1000,91728,549746,NEW HAMPSHIRE,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A strong coastal storm moved up the east coast...,Six to eight inches of snow fell across easter...,CSV
4,201402,13,630,201402,14,800,83476,503982,NEW HAMPSHIRE,33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A significant winter storm brought six to twel...,Five to eight inches of snow fell across easte...,CSV


In [120]:

stormEvents_df = stormEvents_df[['END_YEARMONTH', 'EVENT_ID', 'STATE', 'STATE_FIPS', 'CZ_FIPS', 'CZ_NAME', 'CZ_TYPE', 'YEAR', 'EVENT_TYPE']]
stormEvents_df['CZ_FIPS'] = stormEvents_df['CZ_FIPS'].astype(str)
stormEvents_df['CZ_FIPS'] = '00' + stormEvents_df['CZ_FIPS']
stormEvents_df['CS_FIPS_PADDED'] = stormEvents_df['CZ_FIPS'].str[-3:]



In [121]:
stormEvents_df = stormEvents_df.astype({
    'STATE_FIPS' : str,
    'CS_FIPS_PADDED' : str,
    'END_YEARMONTH' : str})

stormEvents_df['FIPS'] = stormEvents_df['STATE_FIPS'] + stormEvents_df['CS_FIPS_PADDED'] + stormEvents_df['END_YEARMONTH']

In [122]:
stormEvents_df.head(5)

,END_YEARMONTH,EVENT_ID,STATE,STATE_FIPS,CZ_FIPS,CZ_NAME,CZ_TYPE,YEAR,EVENT_TYPE,CS_FIPS_PADDED,FIPS
0,201402,503953,NEW HAMPSHIRE,33,0012,EASTERN HILLSBOROUGH,Z,2014,Heavy Snow,012,33012201402
1,201402,504065,NEW HAMPSHIRE,33,0012,EASTERN HILLSBOROUGH,Z,2014,Heavy Snow,012,33012201402
2,201401,494521,NEW HAMPSHIRE,33,0012,EASTERN HILLSBOROUGH,Z,2014,Heavy Snow,012,33012201401
3,201411,549746,NEW HAMPSHIRE,33,0012,EASTERN HILLSBOROUGH,Z,2014,Heavy Snow,012,33012201411
4,201402,503982,NEW HAMPSHIRE,33,0012,EASTERN HILLSBOROUGH,Z,2014,Heavy Snow,012,33012201402


In [127]:
stormOutages_df = pd.merge(stormEvents_df, outages_df, on='FIPS', how='inner')
stormOutages_df.head(5)

,END_YEARMONTH,EVENT_ID,STATE,STATE_FIPS,CZ_FIPS,CZ_NAME,CZ_TYPE,YEAR,EVENT_TYPE,CS_FIPS_PADDED,FIPS,fips_code,county,state,customers_out,run_start_time,yearMonth
0,201411,543620,SOUTH CAROLINA,45,0079,RICHLAND,C,2014,Thunderstorm Wind,079,45079201411,45079,Richland,South Carolina,50,2014-11-01 08:30:00,201411
1,201411,543620,SOUTH CAROLINA,45,0079,RICHLAND,C,2014,Thunderstorm Wind,079,45079201411,45079,Richland,South Carolina,50,2014-11-01 08:45:00,201411
2,201411,543620,SOUTH CAROLINA,45,0079,RICHLAND,C,2014,Thunderstorm Wind,079,45079201411,45079,Richland,South Carolina,52,2014-11-01 09:00:00,201411
3,201411,543620,SOUTH CAROLINA,45,0079,RICHLAND,C,2014,Thunderstorm Wind,079,45079201411,45079,Richland,South Carolina,52,2014-11-01 09:15:00,201411
4,201411,543620,SOUTH CAROLINA,45,0079,RICHLAND,C,2014,Thunderstorm Wind,079,45079201411,45079,Richland,South Carolina,52,2014-11-01 09:30:00,201411


In [128]:
stormOutages_df = stormOutages_df[['END_YEARMONTH', 'EVENT_ID', 'STATE',  'county', 'YEAR', 'EVENT_TYPE', 'FIPS', 'customers_out']]
stormOutages_df.head(5)

,END_YEARMONTH,EVENT_ID,STATE,county,YEAR,EVENT_TYPE,FIPS,customers_out
0,201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50
1,201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,50
2,201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52
3,201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52
4,201411,543620,SOUTH CAROLINA,Richland,2014,Thunderstorm Wind,45079201411,52


In [129]:
stormOutages_df.to_csv('stormOutages_df.csv', index=False)

In [132]:
# Upload the CSV to S3
s3_client = boto3.client('s3')
s3_client.put_object(Bucket='prj4', Key='stormOutages_df.csv', Body=StringIO().getvalue())

{'ResponseMetadata': {'RequestId': 'PYVMEW8JRPD36YS1',
  'HostId': 'i5WDG8seSqL4UNfhHacBwKlDhGjvJ2PLk/hockTu1Id6+Ukvpkrw3HbqarXuvkQjY2SZyX3gKZM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'i5WDG8seSqL4UNfhHacBwKlDhGjvJ2PLk/hockTu1Id6+Ukvpkrw3HbqarXuvkQjY2SZyX3gKZM=',
   'x-amz-request-id': 'PYVMEW8JRPD36YS1',
   'date': 'Fri, 21 Jun 2024 01:23:10 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
 'ServerSideEncryption': 'AES256'}

In [125]:
stormOutages_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007790 entries, 0 to 1007789
Data columns (total 17 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   END_YEARMONTH   1007790 non-null  object        
 1   EVENT_ID        1007790 non-null  int64         
 2   STATE           1007790 non-null  object        
 3   STATE_FIPS      1007790 non-null  object        
 4   CZ_FIPS         1007790 non-null  object        
 5   CZ_NAME         1007790 non-null  object        
 6   CZ_TYPE         1007790 non-null  object        
 7   YEAR            1007790 non-null  int64         
 8   EVENT_TYPE      1007790 non-null  object        
 9   CS_FIPS_PADDED  1007790 non-null  object        
 10  FIPS            1007790 non-null  object        
 11  fips_code       1007790 non-null  object        
 12  county          1007790 non-null  object        
 13  state           1007790 non-null  object        
 14  customers_out   10

In [ ]:
df.columns

Index(['county', 'state', 'customers_out', 'run_start_time'], dtype='object')